In [1]:
import pandas as pd
import numpy as np

In [96]:
falsa = pd.read_csv(r'/home/roger/Documents/detector_fakenews/scraper/noticia_falsa/csv_novo/boatos.org.csv')

ciencia = pd.read_csv(r'/home/roger/Documents/detector_fakenews/scraper/noticia_verdadeira/csv_novo/ciencia.csv') 
cultura = pd.read_csv(r'/home/roger/Documents/detector_fakenews/scraper/noticia_verdadeira/csv_novo/cultura.csv')
economia = pd.read_csv(r'/home/roger/Documents/detector_fakenews/scraper/noticia_verdadeira/csv_novo/economia.csv')
esportes = pd.read_csv(r'/home/roger/Documents/detector_fakenews/scraper/noticia_verdadeira/csv_novo/esportes.csv')
estilo = pd.read_csv(r'/home/roger/Documents/detector_fakenews/scraper/noticia_verdadeira/csv_novo/estilo.csv')
internacional = pd.read_csv(r'/home/roger/Documents/detector_fakenews/scraper/noticia_verdadeira/csv_novo/internacional.csv')
politica = pd.read_csv(r'/home/roger/Documents/detector_fakenews/scraper/noticia_verdadeira/csv_novo/politica.csv')
tecnologia = pd.read_csv(r'/home/roger/Documents/detector_fakenews/scraper/noticia_verdadeira/csv_novo/tecnologia.csv')

verdadeira = pd.concat([ciencia, cultura, economia, esportes, estilo, internacional, politica, tecnologia])

verdadeira = verdadeira.drop(columns=['id', 'titulo', 'url'])

falsa = falsa.drop(columns=['id', 'link', 'timestamp'])

In [97]:
verdadeira.shape

(3185, 1)

In [98]:
falsa.shape

(1374, 1)

In [99]:
#Removendo números do dataframe

verdadeira['corpo'] = verdadeira['corpo'].str.replace(r'\d+','')

falsa['corpo'] = falsa['corpo'].str.replace(r'\d+', '')

In [100]:
#Transformando letras todas para minúsculas
verdadeira['corpo'] = verdadeira['corpo'].str.lower()

falsa['corpo'] = falsa['corpo'].str.lower()

In [104]:
#removendo ruídos
verdadeira['corpo'] = verdadeira['corpo'].str.replace('(){$%[^\w\s]','')

falsa['corpo'] = falsa['corpo'].str.replace('(){$%[^\w\s]','')

In [105]:
#Mantendo apenas os 100 primeiros caracteres
verdadeira['corpo'] = verdadeira['corpo'].map(lambda x: str(x)[:1000])

In [106]:
verdadeira.shape


(3185, 1)

In [107]:
falsa.shape

(1374, 1)

In [108]:
#Aleatorizando as linhas, para que os assuntos se misturem
verdadeira.sort_values(by='corpo', inplace=True)


In [109]:
#Criando um novo indice apos randomizar as linhas
verdadeira.reset_index(drop=True, inplace=True)

In [110]:
verdadeira.head(10)

,corpo
0,anos da ovelha dolly: por que os clones morre...
1,as atrizes gaia girace (lila) e margherita ma...
2,coisas que fazem você se tornar gay (ou assim...
3,edgar su samsung interrompe a produção do gal...
4,edgar su samsung interrompe a produção do gal...
5,em busca do celular perdidoperder o celular é...
6,foi o ano mais quente de que se tem registroa...
7,horas sem “zap-zap”: brasileiros buscam alter...
8,"horas, transplantes: história de um recorde ..."
9,"jens meyer apacionistas exigem , bilhões de r..."


(3185, 1)

In [111]:
verdadeira_droped = verdadeira.drop(verdadeira.index[1374:])


In [112]:
verdadeira_droped.shape

(1374, 1)

In [113]:
falsa.shape

(1374, 1)

In [114]:
verdadeira_droped.isnull().values.any()

False

In [115]:
falsa.isnull().values.any()

False

In [19]:
import matplotlib.pyplot as plt 

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer as count_vect
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.externals import joblib

import nltk

In [116]:
label_encoder = preprocessing.LabelEncoder()


In [117]:
verdadeira_droped.shape

(1374, 1)

In [118]:
verda = verdadeira_droped.fillna(' ')

In [119]:
verda.shape

(1374, 1)

In [120]:
falsa['label'] = 1


In [121]:
verda['label'] = 0

In [122]:
verda.shape

(1374, 2)

In [123]:
falsa.shape

(1374, 2)

In [128]:
dados = pd.concat([verda,falsa], axis=0)

#y = label_encoder.fit_transform(dados.label.values)
#x = dados.corpo.values

In [130]:
dados

,corpo,label
0,anos da ovelha dolly: por que os clones morre...,0
1,as atrizes gaia girace (lila) e margherita ma...,0
2,coisas que fazem você se tornar gay (ou assim...,0
3,edgar su samsung interrompe a produção do gal...,0
4,edgar su samsung interrompe a produção do gal...,0
5,em busca do celular perdidoperder o celular é...,0
6,foi o ano mais quente de que se tem registroa...,0
7,horas sem “zap-zap”: brasileiros buscam alter...,0
8,"horas, transplantes: história de um recorde ...",0
9,"jens meyer apacionistas exigem , bilhões de r...",0


In [132]:
y = label_encoder.fit_transform(dados.label.values)
x = dados.corpo.values

In [133]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=43, test_size=.33, stratify=y)


In [134]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/roger/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [135]:
pt_stopwords = set(nltk.corpus.stopwords.words('portuguese'))

tfidf = TfidfVectorizer(min_df = 3, strip_accents = 'unicode', max_features = 3000,
                        analyzer = 'word', token_pattern = '\w{1,}',
                        ngram_range = (1,3), sublinear_tf = 1, encoding='utf-8', stop_words = pt_stopwords)



In [136]:
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

In [137]:
classificador = LogisticRegression()
classificador.fit(x_train_tfidf, y_train)

preditor_lr = classificador.predict_proba(x_test_tfidf)

In [139]:
print('Acuracia do modelo Regressão Logística: ',classificador.score(x_test_tfidf, y_test)*100)

Acuracia do modelo Regressão Logística:  99.7794928335171


In [147]:
clf = MultinomialNB().fit(x_train_tfidf,y_train)

preditor_nb = clf.predict(x_test_tfidf)

print('Acuracia do modelo Naive Bayes: ',np.mean(preditor_nb == y_test)*100)


Acuracia do modelo Naive Bayes:  99.55898566703418


In [141]:
clf_svm = svm.SVC(gamma=0.001)
clf_svm.fit(x_train_tfidf, y_train)

clf_svm.predict(x_test_tfidf)

print('Acuracia do modelo SVM: ', clf_svm.score(x_test_tfidf, y_test, sample_weight=None))



Acuracia do modelo SVM:  0.4994487320837927


In [142]:
from sklearn.linear_model import LinearRegression

In [143]:
reg = LinearRegression().fit(x_train_tfidf, y_train)

In [146]:
print('Acuracia do modelo Regressao Linear: ', reg.score(x_train_tfidf, y_train)*100)

Acuracia do modelo Regressao Linear:  99.99999999999433
